In [1]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np

# Time Embedding

https://fridayexperiment.com/how-to-encode-time-property-in-recurrent-neutral-networks/

https://arxiv.org/pdf/1708.00065.pdf

In [63]:
hidden_embedding_size = 10
output_dim = 5

In [64]:
emb_weight = Variable(torch.randn(1, hidden_embedding_size), requires_grad=True)
emb_bias   = Variable(torch.randn(hidden_embedding_size), requires_grad=True)
emb_time   = Variable(torch.randn(hidden_embedding_size, output_dim), requires_grad=True)

emb_weight.shape, emb_bias.shape, emb_time.shape

(torch.Size([1, 10]), torch.Size([10]), torch.Size([10, 5]))

In [4]:
def call(x):
    x = torch.softmax(x * emb_weight + emb_bias, dim=1)
    x = torch.matmul(x, emb_time)
    return x

In [5]:
x = torch.Tensor([1, 2]).unsqueeze(1)

x = call(x)
x.shape

torch.Size([2, 10])

In [6]:
emb_time.shape

torch.Size([5, 10])

In [94]:
class TimeEmbedding(nn.Module):
    def __init__(self, hidden_embedding_size, output_dim):
        super(TimeEmbedding, self).__init__()
        self.emb_weight = nn.Parameter(torch.randn(1, hidden_embedding_size)) # (1, H)
        self.emb_bias = nn.Parameter(torch.randn(hidden_embedding_size)) # (H)
        self.emb_time = nn.Parameter(torch.randn(hidden_embedding_size, output_dim)) # (H, E)

    def forward(self, input):
        # input (B, W, 1)
        x = torch.softmax(input * self.emb_weight + self.emb_bias, dim=2) # (B, W, H)
        x = torch.matmul(x, self.emb_time) # (B, W, E)
        return x

In [95]:
te = TimeEmbedding(50, 10)
x = torch.Tensor([[1, 2, 3], [3,4,5]]).unsqueeze(2)

r = te(x)
print(x.shape)
print(r.shape)

torch.Size([2, 3, 1])
torch.Size([2, 3, 10])


In [33]:
te(x).shape

torch.Size([2, 1, 10])

In [65]:
x.shape, emb_weight.shape

(torch.Size([2, 3, 1]), torch.Size([1, 10]))

In [66]:
x = torch.Tensor([[1, 2, 3], 
                  [3, 4, 5]]).unsqueeze(2)
x.shape

torch.Size([2, 3, 1])

In [71]:
c = (x * emb_weight + emb_bias)
c.shape

torch.Size([2, 3, 10])

In [92]:
c = torch.softmax(c, dim=2)

In [93]:
torch.matmul(c, emb_time).shape

torch.Size([2, 3, 5])

In [72]:
emb_time

tensor([[ 0.2240, -0.7491, -1.0287,  0.4589,  0.1873],
        [-1.1723, -0.2210, -0.7488,  0.5377,  1.5246],
        [ 1.9532,  1.2989, -0.5827,  0.3105, -1.3928],
        [ 0.1894,  0.3690,  0.5805,  0.0558, -1.6729],
        [-1.6623,  0.6312,  0.0351,  0.3948,  1.3067],
        [-1.9284, -0.4763,  0.3184, -1.0626,  0.5354],
        [ 0.1490,  1.0069,  1.1961, -0.2049, -1.4269],
        [-1.0915,  1.2201, -0.5009,  0.7608, -0.8282],
        [-0.4441, -1.7258, -1.5327,  0.1768, -1.3967],
        [-0.5657,  0.8329,  1.1479,  1.8829, -0.8780]], requires_grad=True)